# k-NN
어떤 사람이 다음 대선 때 어떤 후보를 뽑을지 예측해 볼 수 있는가? 어떤 사람에 대해 아무것도 모른다고 하면, 그 사람의 주변 이웃들이 누구를 뽑을지를 살펴보는 것이 좋은 접근일 수 있다. \
만약 그 사람에 대해 아는 것이 거주지, 나이, 소득수준, 자녀 수 등이라고 가정한다면 그 사람의 행동은 이러한 특성들에 의해 어느 정도 영향을 받거나 결정되기 때문에 이 모든 특성을 고려해 가장 가까운 이웃들만을 선별한다면 더 나은 추정을 할 수 있을 것이다. \
이것이 **근접 이웃 분류(nearest neighbors classification)** 기법이다.

## 12.1 모델
k-NN(k-Nearest Neighbors, k-근접 이웃)은 가장 단순한 에측 모델 중 하나이다. 수학적인 가정도 하나 없고 좋은 성능의 컴퓨터가 필요한것도 아니다. \
k-NN을 위해 갖춰야할 요소 \
- 거리를 재는 방법
- 서로 가까운 점들은 유사하다는 가정
다른 알고리즘은 데이터에 내재된 패턴을 찾기 위해 데이터 셋 전체를 봐야 한다. 하지만 k-NN은 궁금한 점 주변에 있는 것만 보면 되서 많은 데이터를 뒤지지 않아도 된다. \
하지만, k-NN은 특정 현상의 원인을 파악하는 데는 큰 도움이 되지 않는다. (어떤 모델들은 나의 투표 결과가 소득 수준이나 혼인 상태에 따라 판가름된다고 이야기해주지만 이웃의 투표 결과에 의해 나의 투표 결과를 판단하는 k-NN은 왜 그렇게 투표를 했는지 설명해주지 않는다.) 

먼저 k를 3또는 5로 정했다고 해보자. 새로운 데이터 포인트를 분휴하고 싶다면 먼저 k개의 가장 가까운 포인트를 찾고, 찾아낸 포인트들의 레이블을 보고 투표를 통해 새로운 데이터 포인트의 레이블을 정할 수 있다. 이렇게 하기 위해서 먼저 투표를 집계하는 함수를 다음과 같이 만들업 보자. 

In [1]:
from typing import List
from collections import Counter

def raw_majority_vote(labels: List[str]) -> str:
    votes = Counter(labels)
    winner, _ = votes.most_common(1)[0]
    return winner

assert raw_majority_vote(['a', 'b', 'c', 'b']) == 'b'

위와 같이 함수를 짜게되면 동점인 항목들을 똑똑하게 처리가 되지 않는다. 예를 들면 영화에 등급을 매긴다면, 가장 인접한 영화가 각각 전체 관람가, 전체 관람가, 13세 관람가, 13세 관람가, 19세 관람가로 구분된다. 전체 관람가 2개, 13세 관람가 2개로 공동 1등이 되는데, 이 경우 아래와 같이 조치 한다. \
- 여러 1등 중 임의로 하나를 정한다.
- 거리를 가중치로 사용하여 이를 고려한 투표를 한다.
- 단독 1등이 생길 때까지 k를 하나씩 줄인다.

단독 1등이 생길 때까지 k를 하나씩 줄인다룰 구현해보자

In [3]:
def majority_vote(labels: List[str]) -> str:
    """labels는 가장 가까운 데이터부터 가장 먼 데이터 순서로 정렬되어 있다고 가정"""
    vote_counts = Counter(labels)
    winner, winner_count = vote_counts.most_common(1)[0]
    num_winners = len([count for count in vote_counts.values()
                      if count == winner_count])
    if num_winners == 1:
        return winner
    else:
        return majority_vote(labels[:-1])  # 가장 먼 데이터를 제외하고 다시 찾아본다.
    
assert majority_vote(['a', 'b', 'c', 'b', 'a']) == 'b'

In [7]:
labels = ['a', 'b', 'c', 'b', 'a']
vote_counts = Counter(labels)
print(vote_counts)
print(vote_counts.most_common())  # list로 변환
print(vote_counts.most_common(1))  # list 에서 하개만 추출
print(vote_counts.most_common(1)[0])

Counter({'a': 2, 'b': 2, 'c': 1})
[('a', 2), ('b', 2), ('c', 1)]
[('a', 2)]
('a', 2)


가장 극단적인 경우 k가 1이 돼서 가장 가까운 레이블 하나만 보고 새로운 레이블을 정하면 된다.

In [8]:
from typing import NamedTuple, List

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """v_1 * w_1 + ... + v_n * w_n"""
    assert len(v) == len(w),  "vectors must be same length"
    
    return sum(v_i * w_i for v_i, w_i in zip(v,w))

def sum_of_squares(v: Vector) -> float:
    """v_1 * v_1 + ... v_n * v_n"""
    return dot(v,v)

def squared_distance(v: Vector, w: Vector) -> float:
    """(v_1 - w_1)**2 + ... + (v_n - w_n)**2"""
    return sum_of_squares(subtract(v, w))

def distance(v: Vector, w: Vector) -> float:
    """벡터 v와 w 간의 거리를 계산"""
    return math.sqrt(squared_distance(v,w))

In [10]:
class LabeledPoint(NamedTuple):
    point: Vector
    label: str

def knn_classify(k: int,
                labeled_points: List[LabeledPoint],
                new_point: Vector) -> str:
    # 레이블된 포인트를 가장 가까운 데이터부터 가장 먼 데이터 순서로 정렬
    by_distance = sorted(labeled_points,
                        key = lambda lp: distance(lp.point, new_point))
    
    # 가장 가까운 k 데이터 포인트의 레이블을 살펴보고
    k_nearest_labels = [lp.label for lp in by_distance[:k]]
    
    # 투표한다.
    return majority_vote(k_nearest_labels)

## 12.2 예시: Iris 데이터
Iris 데이터는 기계학습에서 자주 쓰이는 데이터셋 중 하나이다. Iris 데이터는 붓꽃(Iris) 세 종에 속하는 150개의 꽃의 다양한 측정치를 담고 있다. \
각각 꽃의 길이(lenght), 너비(width), 꽃바침의 길이(sepal length), 꽃바침 너비(sepal width), 종(species)가 기록되어 있다. \

In [12]:
import requests

data = requests.get("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data")

In [14]:
with open('iris.data', 'w') as f:
    f.write(data.text)

In [15]:
from typing import Dict
import csv
from collections import defaultdict

def parse_iris_row(row: List[str]) -> LabeledPoint:
    """
    꽃바침 길이, 꽃바침 너비, 꽃잎 길이, 꽃잎 너비, 분류
    """
    measurements = [float(value) for value in row[:-1]]
    # 분류는 "Iris-virginica와 같이 나오는데 그중 "verginica"만 뽑자
    label = row[-1].split("-")[-1]
    return LabeledPoint(measurements, label)    

In [20]:
with open("iris.data") as f:
    reader = csv.reader(f)
    iris_data = [parse_iris_row(row) for row in reader]

IndexError: list index out of range

In [24]:
# 데이터를 살펴보기 위해 종/레이블로 무리를 지어보자.
points_by_species: Dict[str, List[Vector]] = defaultdict(list)
for iris in iris_data:
    points_by_species[iris.label].append(iris.point)

NameError: name 'iris_data' is not defined

해당 데이터는 4차원이기 때문에 그려 보기 쉽지 않다. 따라서 산점도(scatterplot)를 살펴보자. 

In [25]:
import matplotlib.pyplot as plt
metrics = ['sepal length', 'sepal width', 'petal length', 'petal width']
pairs = [(i, j) for i in range(4) for j in range(4) if i < j]
marks = ["+", ".", "x"]

fig, ax = plt.subplot(2,3)
for row in range(2):
    for col in range(3):
        i, j = pairs[3 * row + col]
        ax[row][col].set_title(f"{metrics[i]} vs {metrics[j]}", fontsize=8)
        ax[row][col].set_xticks([])
        ax[row][col].set_yticks([])
        
        for mark, (species, points) in zip(marks, points_by_species.items()):
            xs = [point[i] for point in points]
            ys = [point[j] for point in points]
            ax[row][col].scatter(xs, ys, marker=mark, label=species)

ax[-1][-1].legend(loc='lower right', prop={'size': 6})
plt.show()

ValueError: Illegal argument(s) to subplot: (2, 3)

<Figure size 432x288 with 0 Axes>

In [26]:
# 학습 데이터와 평가 데이터로 나눈다.
import random
from typing import TypeVar, List, Tuple

X = TypeVar('X')  # 데이터를 표현하기 위한 일반적인 타입

def split_data(data: List[X], prob: float) -> Tuple[List[X], List[X]]:
    """데이터를 [prob, 1 - prob]의 비율로 나눈다."""
    data = data[:]  # 얕은 복사본을 만든다.
    random.shuffle(data)  # shuffle이 리스트 내용을 바꾸기 때문
    cut = int(len(data) * prob)  # prob을 사용하여 자를 위치를 선택하고
    return data[:cut], data[cut:]  # 섞인 리스트를 자른다.

In [27]:
random.seed(12)
iris_train, iris_test = split_data(iris_data, 0.70)
assert len(iris_train) == 0.7 * 150
assert len(iris_test) == 0.3 * 150

NameError: name 'iris_data' is not defined

학습 데이터가 '이웃' 역할을 하여 평가 데이터를 분휴하게 될것이다. 투표를 위해 이웃의 숫자인 k 값을 결정해야 한다. \
만약, k가 1처럼 너무 작으면 이상치가 결과에 너무 큰 영향을 주게 될 것이다. \
반면, k가 너무 크다면 데이터 안에서 가장 빈번하게 발생한 클래스가 선택될 것이다. \
현실에서는 k-NN을 활용할 때는 별도의 검증 데이터를 통해서 k를 찾을 수도 있다. 여기서는 k=5로 사용하자.

In [28]:
from typing import Tuple

# 우리가 (predicted, actual)을 몇 번 살펴보는지 추적하자.
confusion_matrix: Dict[Tuple[str, str], int] = defaultdict(int)
num_correct = 0

for iris in iris_test:
    predicted = knn_classify(5, iris_train, iris_test)
    actual = iris.label
    
    if predicted == actual:
        num_correct += 1
    
    confusion_matrix((predicted, actual)) += 1
    
pct_correct = num_correct / len(iris_test)
print(pct_correct, confusion_matrix)

SyntaxError: cannot assign to function call (<ipython-input-28-877ca6dd056b>, line 14)

## 12.3 차원의 저주
k-NN은 '차원의 저주(curse of dimensionality)'라는 것 때문에 고차원에서 문제가 생긴다. \
고차원 공간은 엄청나게 넓기 때문에 고차원에서 데이터들은 서로 '근접'하지 않게 된다. 이 현상을 관찰하기 위해서는 d-차원 단위 정육면체에 안에서 임의의 점 두 개를 생성해 보며 차원을 늘려 보자.

In [29]:
def random_point(dim: int) -> Vector:
    return [random.random() for _ in range(dim)]

In [30]:
def random_distances(dim: int, num_pairs: int) -> List[float]:
    return [distance(random_point(dim), random_point(dim)) for _ in range(num_pairs)]

1차원부터 100차원까지 각각의 차원에 대해 총 10,000개의 거리를 계산한 뒤, 각 점들 간의 평균 거리와 최소 거리를 구해보자

In [33]:
def subtract(v: Vector, w: Vector) -> Vector:
    """각 성분끼리 뺀다."""
    assert len(v) == len(w), "vectors must be the same length"
    
    return [v_i - w_i for v_i, w_i in zip(v, w)]

In [34]:
import math
import tqdm
dimensions = range(1, 100)

avg_distances = []
min_distances = []

random.seed(0)
for dim in tqdm.tqdm(dimensions, desc="Curse of Dimensionality"):
    distances = random_distances(dim, 10000)  # 10,000개 임의의 쌍
    avg_distances.append(sum(distances) / 10000)  # 평균 거리
    min_distances.append(min(distances))  # 최소 거리를 저장

Curse of Dimensionality: 100%|██████████| 99/99 [00:14<00:00,  6.73it/s]


차원이 증가할수록 점들 간 평균 거리도 증가하지만, 더 큰 문제는 최근접 거리와 평균 거리의 비율이다.

In [35]:
min_avg_ratio = [min_dist / avg_dist
                for min_dist, avg_dist in zip(min_distances, avg_distances)]

저저차원에서는 근접 이웃들이 평균 거리에 비해 월등히 가깝다. 하지만 두 점이 '가깝다'고 하려면 모든 차원에 관해 가까워야 한다. 차원이 추가된다는 것은 두 점이 가까울 수 있는 가능성이 현저히 줄어든다는 것을 의미하기도 한다. 즉, 고차원일 때는 근접 이웃들이 평균 거리와 큰 차이가 나지 않게 되고, 그렇게 때문에 가깝다는 것이 별 의미를 가지지 않게 된다.

고차원 문제를 바라볼 수 있는 또 다른 관점은 공간의 성김(sparsity)과 관련이 있다. \
0~1 사이에서 50개의 점을 임의로 고르면 단위 구간을 제법 채울 수 있다. \
2차원상에서 똑같이 50개를 고르면 단위 정사각형을 덜 채우게 된다. \
3차원에서는 그보다 더 적은 공간을 채우게 된다. \
matplotlib로는 4차원을 그릴 수 없다. 
**차원이 높아질수록 점들 사이의 거리의 거리가 멀어지고 빈 공간이 많아진다.** \
데이터가 기하급수적으로 많아지지 않으면, 고차원에서의 빈 공간은 우리가 예측하고 싶은 값을 포함한 모든 점에게 공통적으로 먼 공간이 된다. \
$\rightarrow$ 고차원에서 k-NN을 이용하려면 먼저 차원 축소를 하는 것이 좋을 것이다.!